In [129]:
import os, lhapdf
from shutil import copyfile

In [130]:
lhapdf_path = '/opt/lhapdf6/share/LHAPDF/'
new_set = 'NNPDF30_nnlo_as_0118_qed_toevolve'
pdf_prior = 'NNPDF30_nnlo_as_0118'
pdf_photon_prior = 'NNPDF23_nnlo_as_0118_qed'

In [131]:
pdf = lhapdf.mkPDFs(pdf_photon_prior)

In [132]:
try: os.mkdir(new_set)
except: pass
copyfile(lhapdf_path + pdf_prior + '/' + pdf_prior + '.info', new_set + '/' + new_set + '.info')

In [133]:
for mem, ipdf in enumerate(pdf):
    f = open(lhapdf_path + pdf_prior + '/' + pdf_prior + '_%04d.dat' % mem,'r')
    lines = f.readlines()
    f.close()

    x = [ float(item) for item in lines[3].split() ]
    q = []
    q.append([ float(item) for item in lines[4].split()])
    q.append([ float(item) for item in lines[408].split()])
    q.append([ float(item) for item in lines[1612].split()])

    pht = []
    for sbq in q:
        for ix in x:
            for iq in sbq:
                if iq < 2**0.5: iq = 2**0.5 # vinculate to 2**0.5
                pht.append('%.7e' % ipdf.xfxQ(22, ix, iq))   

    out = open(new_set + '/' + new_set + '_%04d.dat' % mem,'w')          
    index = 0
    for id, line in enumerate(lines):
        if id < 5 or (id >= 406 and id < 409) or (id >= 1610 and id < 1613) or id == 5014: pass
        elif id == 5 or id == 409 or id == 1613: line= line.replace('\n','') + ' 22\n'
        else:
            line = line.replace('\n','  ') + pht[index] + '\n'
            index+=1
        out.write(line)
    out.close()